# Zadanie 1.

In [ ]:
function harmonic_mean_gen_impl(x...)
    N = length(x)
    elNum = :($N)
    sum = :(0)
    for i = 1:N
        sum = :((1 / x[$i]) + $sum)
    end
    return :($elNum / $sum)
end

In [ ]:
harmonic_mean_gen_impl(Int, Int, Int)

In [ ]:
harmonic_mean_gen_impl(2, 2, 5, 7)

# Zadanie 2.

In [ ]:
dump(:(x + x))

In [ ]:
function autodiff(ex::Expr)::Expr
    if (ex.head == :call) autodiff(Val{ex.args[1]}, ex) end
end

In [ ]:
function autodiff(n::Number) return 0 end
function autodiff(s::Symbol) return 1 end

In [ ]:
function autodiff(::Type{Val{:+}}, ex::Expr)::Expr
    reduce((x,y) -> :($x + $y), map(autodiff, ex.args[2:end]))
end

function autodiff(::Type{Val{:-}}, ex::Expr)::Expr
    reduce((x,y) -> :($x - $y), map(autodiff, ex.args[2:end]))
end

function autodiff(::Type{Val{:*}}, ex::Expr)
    res = :(0)
    args = ex.args[2:end]
    argsdiff = map(autodiff, args)
    N = length(args)
    
    for i = 1:N
        part = 1
        for j = 1:N
            if i == j 
                ad = argsdiff[j]
                part = :($part * $ad)
            else 
                a = args[j]
                part = :($part * $a)
            end
        end
        res = :($res + $part)
    end
    res
end

function autodiff(::Type{Val{:/}}, ex::Expr)
    f = ex.args[2]
    g = ex.args[3]
    df = autodiff(ex.args[2])
    dg = autodiff(ex.args[3])
    :(($df * $g - $f * $dg) / ($g * $g))
end

In [ ]:
autodiff(:(x+2))

In [9]:
autodiff(:(x-2))

:(1 - 0)

In [10]:
autodiff(:(5 * x * x))

:(((0 + ((1 * 0) * x) * x) + ((1 * 5) * 1) * x) + ((1 * 5) * x) * 1)

In [11]:
autodiff(:(5 / x + x * y))

:((0x - 5 * 1) / (x * x) + ((0 + (1 * 1) * y) + (1x) * 1))

In [12]:
autodiff(:(x + x*y*z))

:(1 + (((0 + ((1 * 1) * y) * z) + ((1x) * 1) * z) + ((1x) * y) * 1))

## Zadanie bonusowe.

In [13]:
using DualNumbers
function autodiffFun(f) 
    g(x) = dualpart(f(Dual(x,1)))
end

autodiffFun (generic function with 1 method)

In [14]:
f(x) = x * x + x - 10x
g = autodiffFun(f)
g(5) == 1

true